In [1]:
class Region():
    def __init__(self, crop):
        self.crop = crop
        self.perimeter = 0
        self.children = set()
        self.edges = set()

    def __repr__(self):
        return f'Region ({self.crop}) of size {len(self.children)} with perimeter {self.perimeter} - score {len(self.children) * self.perimeter}'

    def add_child(self, n):
        self.children.add(n)

In [2]:
# with open("simple_test.txt") as fh:
# with open("4test_input.txt") as fh:
# with open("etest.txt") as fh:
# with open("test_input.txt") as fh:
with open("input.txt") as fh:
    raw_data = [x.strip() for x in fh.readlines()]

In [3]:
data = []
for y, row in enumerate(raw_data):
    newcol = []
    for x, col in enumerate(row):
        newcol.append(col)
    data.append(newcol)

In [4]:
from collections import deque, defaultdict

In [5]:
segmented = set()
cur_region = False
regions = []

def find_plot_members(root, region: Region):
    queue = deque()
    seen = defaultdict(set)
    done = set()
    x, y = root
    queue.append((x, y))
    while len(queue) > 0:
        x, y = queue.popleft()
        for xdir, ydir in [[-1, 0], [0, 1], [0, -1], [1, 0]]:
            new_x = x + xdir
            new_y = y + ydir
            if new_x < 0 or new_x > len(data[0]) - 1 or new_y < 0 or new_y > len(data) - 1:
                region.perimeter += 1
                region.edges.add((new_x, new_y))
                continue
            new_crop = data[new_y][new_x]
            if (x, y) in seen[(new_x, new_y)] or (new_x, new_y) in seen[(x, y)]:
                 continue
            seen[(new_x, new_y)].add((x, y))
            seen[(x, y)].add((new_x, new_y))
            if new_crop != data[y][x]:
                region.perimeter += 1
                region.edges.add((new_x, new_y))
            else:
                region.add_child((new_x, new_y))
                segmented.add((new_x, new_y))
                if (new_x, new_y) not in done:
                    queue.append((new_x, new_y))
                done.add((new_x, new_y))


for y, row in enumerate(data):
    for x, col in enumerate(row):
        if (x, y) in segmented:
            continue
        segmented.add((x, y))
        cur_region = Region(col)
        cur_region.add_child((x, y))
        find_plot_members((x, y), cur_region)
        regions.append(cur_region)




In [6]:
res = 0
for r in regions:
    res += len(r.children) * r.perimeter
print(res)

1361494


In [7]:
def get_num_sides(region):
    corners = 0
    if len(region.children) <= 2:
        return 4
    for x, y in region.children:
        children_connections = 0
        for xdir, ydir in [[-1, 0], [0, 1], [0, -1], [1, 0]]:
            if (x + xdir, y + ydir) in region.children:
                children_connections += 1
        if children_connections == 1:
            corners += 2
            continue
        #  top left
        if (((x - 1, y) in region.children and (x, y - 1) in region.children) and ((x - 1, y - 1) not in region.children)) or (
            ((x - 1, y) not in region.children and (x, y - 1) not in region.children)
        ):
            corners += 1
        #  top right
        if (((x + 1, y) in region.children and (x, y - 1) in region.children) and ((x + 1, y - 1) not in region.children)) or (
            ((x + 1, y) not in region.children and (x, y - 1) not in region.children)
        ):
            corners += 1
        #  bottom left
        if (((x - 1, y) in region.children and (x, y + 1) in region.children) and ((x - 1, y + 1) not in region.children)) or (
            ((x - 1, y) not in region.children and (x, y + 1) not in region.children)
        ):
            corners += 1
        #  bottom right
        if (((x + 1, y) in region.children and (x, y + 1) in region.children) and ((x + 1, y + 1) not in region.children)) or (
            ((x + 1, y) not in region.children and (x, y + 1) not in region.children)
        ):
            corners += 1
    return corners

tot = 0
for r in regions:
    num_sides = get_num_sides(r)
    tot += len(r.children) * num_sides
print(tot)

830516
